In [1]:
import pandas as pd 
import re
import nltk
from nltk.corpus import stopwords 
import string
import itertools
import networkx as nx 
import datetime as dt
import pickle5 as pickle
from itertools import chain
import numpy as np

# DISCLAIMER
# change env to python 3.8 (python 3.7 (default) -> Error: Unsupported Pickle Protocol 5)

In [2]:
path="./DATA/"
path_to_protocol5=path+"Tweets/REDFLAG_2021_09_01_to_2021_12_31.pkl"
with open(path_to_protocol5, "rb") as fh:
    tweets_df = pickle.load(fh)

In [3]:
tweets_df.columns = ['Datetime', 'Tweet Id', 'Text', 'Username', "Hashtags"]
tweets_df = tweets_df[tweets_df["Datetime"] < "2021-12-1"]

In [4]:
tweets_filtered = tweets_df.copy() #it's a good idea to work on the copy of original dataframe, so we can always go back to it if we mess something up
tweets_filtered.tail()

,Datetime,Tweet Id,Text,Username,Hashtags
6483,2021-09-01 02:44:50+00:00,1432897207326285826,@TabbyCa25757103 @Walkabout24 It is the bigges...,TracyMMPohill,[RedFlag]
6484,2021-09-01 02:08:41+00:00,1432888109696524288,#Boma is a serious #redflag -ladies all over t...,Vivscorner_,"[Boma, redflag, Tega, BBNajia6]"
6485,2021-09-01 01:12:05+00:00,1432873864032399360,@amundsenio unintentionally funny #redFlag 🚩 h...,billypuntove,[redFlag]
6486,2021-09-01 00:59:24+00:00,1432870673710473217,I’m stuck on the fact she tracking my step son...,SwaqqaBee,[RedFlag]
6487,2021-09-01 00:52:09+00:00,1432868848299020290,2 senior FDA Vax regulators are stepping down ...,turn2asavage,[redflag]


## NLTK tools

In [5]:
import re #for regex matching
import nltk #natural language toolkit library
from nltk.corpus import wordnet as wn #importing it 
from nltk.stem.wordnet import WordNetLemmatizer #importing wordnet lemmatizer
from nltk import pos_tag #part-of-speech-tagger
from collections import defaultdict #defaultdict returns default value for non-existant keys you try to  access based on the function you passed in the constructor

In [6]:
#nltk.download('words')
#nltk.download('stopwords') #downloading stopwords
#nltk.download('wordnet') #downloading wordnet
#nltk.download('averaged_perceptron_tagger') #downloading tagger
#nltk.download('omw-1.4')

In [7]:
words = set(nltk.corpus.words.words()) #the last two lines serve to download the corpus of standard English language words
stop_words = set(nltk.corpus.stopwords.words("english")) #taking the stop words from English language
tag_map = defaultdict(lambda : wn.NOUN) #here we define that wn.NOUN is the default value for the dict
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [8]:
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","", tweet) # remove mentions (not interested in them)
    tweet = re.sub("#[A-Za-z0-9]+", "", tweet) # remove hashtags (we have them in a specific column)
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) # remove http links (not interested in them)
    tweet = " ".join(tweet.split())
    tweet = str.lower(tweet) #to lowercase
    lemma_function = WordNetLemmatizer()
    #basically we use pos_tag function on tokens that we get by applying wordpunct tokenization
    #to tweet (it separates all the words and symbols)
    #then we pass the token along with it's wordnet pos value that we get from the tag_map dictionary 
    #(noun, adjective, verb or adverb) to the lemma function (the WordNetLemmatizer()) 
    tweet = " ".join(lemma_function.lemmatize(token, tag_map[tag[0]]) 
                     for token, tag in nltk.pos_tag(nltk.wordpunct_tokenize(tweet))) #lemmatize
    
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
    if w.lower() in words and not w.lower() in stop_words) #remove stop words

    tweet = str.lower(tweet) #to lowercase
    return tweet

tweets_filtered["clean_text"] = tweets_filtered["Text"].map(cleaner)

In [9]:
def cleanu (tweet):
    tweet = tweet.split()
    for i in range(len(tweet)):
        if tweet[i] == "u":
            tweet[i] = "you"
    tweet = " ".join(w for w in tweet)
    return tweet

tweets_filtered["clean_text"] = tweets_filtered["clean_text"].map(cleanu)

In [10]:
unique_hashtags = {}

for idx, row in tweets_filtered.iterrows():
    hashtag_list = []
    for hashtag in row["Hashtags"]:
        hashtag = hashtag.lower()
        if hashtag == "redflags":
            hashtag = "redflag"
        if hashtag == "redflags" or hashtag == "trending" :
            hashtag = "redflag"
        elif hashtag == "redflagtrend" or hashtag == "redflagstrend":
            hashtag = "redflag"
        elif  hashtag == "redflagschallenge" or hashtag == "redflagchallenge" or hashtag=="redflagwarning":
            hashtag = "redflag"
            
        unique_hashtags.setdefault("#"+hashtag, 0)
        unique_hashtags['#'+hashtag] += 1
        hashtag_list.append(hashtag)
        tweets_filtered.at[idx, "hashtags"] = hashtag_list
unique_hashtags.pop('#redflag') # remove redflag

7048

In [11]:
# hashtags_list = list(unique_hashtags.keys())
# for i in range(len(hashtags_list)):
#     hashtags_list[i] = hashtags_list[i][1:]

In [12]:
# # Procedure to filtrate only tweets with hashtag inside the community
# tweets_filtered_unique = tweets_filtered.copy().reset_index()
# hashtags_list_list = []
# idx_list = []
# for c in hashtags_list:
#     idx = list(tweets_filtered_unique[tweets_filtered_unique['hashtags'].apply(lambda x: c in x)].index)
#     l = len(np.unique(tweets_filtered_unique.iloc[list(idx)]['Username']))
#     if l > 3:
#         hashtags_list_list.append(c)

In [13]:
# tweets_filtered[tweets_filtered['hashtags'].apply(lambda x: 'advice' in x)]

In [14]:
# # Procedure to filtrate only tweets with hashtag inside the community
# tweets_filtered = tweets_filtered.copy().reset_index()
# idx_list = []
# for c in hashtags_list_list:
#     idx_list.append(list(tweets_filtered[tweets_filtered['hashtags'].apply(lambda x: c in x)].index))

# idx = np.unique(list(chain(*idx_list)))
# idx
# tweets_filtered = tweets_filtered.iloc[list(idx)]
# tweets_filtered

# Hashtag Analysis

Dict of unique hashtag without 'redflag'

In [15]:
# unique_hashtags = {}

# for idx, row in tweets_filtered.iterrows():
#     hashtag_list = []
#     for hashtag in row["Hashtags"]:
#         hashtag = hashtag.lower()
#         if hashtag in hashtags_list_list:
#             unique_hashtags.setdefault("#"+hashtag, 0)
#             unique_hashtags['#'+hashtag] += 1
#             hashtag_list.append(hashtag)
#             tweets_filtered.at[idx, "hashtags"] = hashtag_list

In [16]:
# len(hashtags_list_list)

In [17]:
# len(unique_hashtags)

In [18]:
uh_df = pd.DataFrame.from_dict(unique_hashtags, orient='index').reset_index()
uh_df.rename(columns = {'index':'Hashtag', 0:'Count'}, inplace=True)
uh_df.sort_values(by=['Count'], ascending=False, inplace=True)
uh_df

,Hashtag,Count
755,#redflagmeme,71
4123,#datingadvice360,69
987,#datingadvice,67
1487,#datingtips,62
386,#f1,62
...,...,...
1971,#themarque,1
1972,#onlinepersona,1
1973,#crystalball,1
1974,#dogwalker,1


## Network Creation

In [ ]:
#uh = unique_hashtags.keys()
#uw = unique_words.keys()

### Hashtags Network

In [32]:
# Variable for filtering
h = 0

In [20]:
network = {} 
network_key = 0
for index, row in tweets_filtered.iterrows():
    #hashtags extracted from Tweet do not have the # sign in front of them but we will add it to differentiate hashtags from words
    combined_list = ['#'+hashtag for hashtag in row["hashtags"] if '#'+hashtag in unique_hashtags]
    #itertool product creates Cartesian product of each element in the combined list (hastag + words, both UNIQUE)
    for pair in itertools.product(combined_list, combined_list):
        #exclude self-loops and count each pair only once because our graph is undirected and we do not take self-loops into account
        if pair[0]!=pair[1] and not(pair[::-1] in network):
            network.setdefault(pair, 0)
            network[pair] += 1 

network = {key: value for (key, value) in network.items() if value >= h} #filter the network
network_df = pd.DataFrame.from_dict(network, orient="index")

In [21]:
network_df.reset_index(inplace=True) 
network_df.columns = ["pair","weight"] # undirected graph 
network_df.sort_values(by="weight", inplace=True, ascending=False) 
network_df

,pair,weight
625,"(#f1, #dutchgp)",47
35,"(#crypto, #bitcoin)",20
627,"(#dutchgp, #zandvoort)",15
23,"(#podcast, #dating)",10
646,"(#zandvoort, #hamilton)",10
...,...,...
272,"(#twitter, #tweet)",1
271,"(#relationships, #tweet)",1
270,"(#relationships, #twitter)",1
269,"(#domesticviolence, #relationships)",1


In [22]:
#to get weighted graph we need a list of 3-element tuplels (u,v,w) where u and v are nodes and w is a number representing weight
up_weighted = []
for edge in network:
    #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
    #if(network[edge])>1: 
    up_weighted.append((edge[0],edge[1],network[edge]))

G = nx.Graph()
G.add_weighted_edges_from(up_weighted)

In [23]:
print(len(G.nodes())) # number of nodes
print(len(G.edges()))

180
665


In [24]:
hashtag_nodes = uh_df.copy()
hashtag_nodes = hashtag_nodes.drop(hashtag_nodes[hashtag_nodes['Hashtag'] == '#redflag'].index)
hashtag_nodes["Label"] = hashtag_nodes["Hashtag"]
hashtag_nodes.rename(columns={"Hashtag":"Id"},inplace=True)
hashtag_nodes = hashtag_nodes.drop(columns=['Count'])
hashtag_nodes

,Id,Label
86,#redflagmeme,#redflagmeme
104,#datingadvice,#datingadvice
55,#f1,#f1
185,#dutchgp,#dutchgp
3,#dating,#dating
...,...,...
39,#respectfully,#respectfully
152,#technology,#technology
153,#accessibility,#accessibility
156,#tweet,#tweet


Number of nodes and number of unique_hashtags are different due to filter procedure. We only keep nodes.

In [25]:
from cdlib import algorithms, viz
coms = algorithms.louvain(G, weight='weight', resolution=1., randomize=False)
#coms = algorithms.surprise_communities(G)
#coms = algorithms.leiden(G)
#coms = algorithms.walktrap(G)

In [26]:
hashtag_nodes['Modularity Class'] = 0

k = 1
for community in coms.communities:
    for hashtag in hashtag_nodes['Label']:
        if hashtag in community:
            idx = hashtag_nodes[hashtag_nodes['Label'] == hashtag].index
            hashtag_nodes['Modularity Class'][idx] = k
    k += 1

hashtag_nodes = hashtag_nodes.drop(hashtag_nodes[hashtag_nodes['Modularity Class'] == 0].index) # this is for remove nodes without edges

D:\anaconda3\envs\network\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [27]:
# lunghezze = []
# for i in range(len(np.unique(hashtag_nodes['Modularity Class']))):
#     lunghezze.append(len(hashtag_nodes[hashtag_nodes['Modularity Class'] == i]))

In [28]:
# hashtag = np.array(hashtag_nodes[hashtag_nodes['Modularity Class'] == 1]['Id'])

Node_list save procedure for Gephi visualization:

In [29]:
algo = '3'

In [30]:
# SAVE NODE LIST
hashtag_nodes.to_csv("./DATA_COMM/user/#nodelist_community_" + algo +".csv", index=False)

# We need to insert a column called "Modularity Class" and label
file = pd.read_csv("./DATA_COMM/user/#nodelist_community_" + algo +".csv", sep = ",", header = 0, names = ['Id','Modularity Class'])
file.insert(loc = 1, column = 'Label', value = file['Id'])
file.to_csv("./DATA_COMM/user/#nodelist_community_" + algo +".csv", index=False)

Edge_list saving procedure for Gephi visualization:

In [31]:
nx.write_weighted_edgelist(G, "./DATA_COMM/user/#edgelist_community_" + algo +".csv")

# We need to insert source, target, weight and type of edge
file = pd.read_csv("./DATA_COMM/user/#edgelist_community_" + algo +".csv", sep = " ", header = 0, names = ['Source','Target','Weight'])
file.insert(loc = 2, column = 'Type', value = ['undirected']*len(file))
file.to_csv("./DATA_COMM/user/#edgelist_community_" + algo +".csv", index=False)

### hashtag_nodes

# 2 cells for select only the tweets with the selected hashtags
### Let's divide in community. In this case we choose the first one.
Each modularity class is a community

In [19]:
# hashtags_list = list(hashtag_nodes[hashtag_nodes['Modularity Class'] == 1]['Id'])
# for i in range(len(hashtags_list)):
#     hashtags_list[i] = hashtags_list[i][1:]

In [20]:
# len(hashtags_list)

In [94]:
# hashtags_list = ['covid19',
#                  'gaslighting',
#                  'plandemic',
#                  'freedom',
#                  'vaccines',
#                  'truthmatters',
#                  'covid',
#                  'nonprofit',
#                  'antivaxxers',
#                  'community',
#                  'education',
#                  'science ']

# hashtags_list = ['climatechange',
#                  'climatecrisis',
#                  'suistainable',
#                  'savetheplanet',
#                  'zerowaste',
#                  'climatecrisis',
#                  'climate']

# hashtags_list = ['domesticviolence',
#                  'domesticabuse',
#                  'coercivecontrol',
#                  'abuse',
#                  'guncontrol',
#                  '2a',
#                  'guns',
#                  'gunsense',
#                  'redflaglaws']

# hashtags_list = ['mentalhealth',
#                  'health',
#                  'feminism',
#                  'toxic',
#                  'mentalhealthmatters',
#                  'selfcare',
#                  'metoo',
#                  'facts',
#                  'girls',
#                  'motivation',
#                  'mentalhealthawereness',
#                  'selflove',
#                  'depression',
#                  'run',
#                  'meme',
#                  'memes',
#                  'blacktwitter',
#                  'life',
#                  'selfguarding',
#                  'swipeleft',
#                  'tinder',
#                  'wtf',
#                  'mindfulness',
#                  'awereness',
#                  'cannabiscommunity',
#                  'cbd']

# hashtags_list = ['dating',
#                  'relationships',
#                  'relationshipadvice',
#                  'reallife',
#                  'relatable',
#                  'flag',
#                  'stayway',
#                  'funny',
#                  'humor',
#                  'datingadvice',
#                  'datingtips',
#                  'comedy',
#                  'women',
#                  'men',
#                  'datingredflags',
#                  'alert',
#                  'lgtb',
#                  'horoscope']

hashtags_list = ['business',
                 'digitalmarketing',
                 'fraud',
                 'trading',
                 'money',
                 'advice',
                 'facebook',
                 'twitter',
                 'trend',
                 'linkedin',
                 'instagram',
                 'entrepreneur']

In [95]:
tweets_comm = tweets_filtered.copy().reset_index()

In [96]:
# Procedure to filtrate only tweets with hashtag inside the community
tweets_comm = tweets_filtered.copy().reset_index()
idx_list = []
for c in hashtags_list:
    idx_list.append(list(tweets_comm[tweets_comm['hashtags'].apply(lambda x: c in x)].index))

idx = np.unique(list(chain(*idx_list)))
idx
tweets_comm = tweets_comm.iloc[list(idx)]
tweets_comm

,index,Datetime,Tweet Id,Text,Username,Hashtags,clean_text,hashtags
55,989,2021-11-28 01:26:24+00:00,1464767600597082117,5 Red flags to watch out for at the beginning ...,RuthlessCI,"[Relationship, RedFlag, Love, Advice, FairyTale]",red flag watch beginning relationship read ful...,"[relationship, redflag, love, advice, fairytale]"
207,1141,2021-11-22 15:30:18+00:00,1462805647754514432,#BadData is no good. We at Merrill Research id...,ResearchMerrill,"[BadData, data, business, RedFlag]",good research identify bad evaluate individual...,"[baddata, data, business, redflag]"
238,1172,2021-11-21 01:26:22+00:00,1462230876683710467,Nobody's perfect.\n\nRead the full article: 5 ...,RuthlessCI,"[Relationship, RedFlag, Love, Advice, FairyTale]",nobody perfect read full article reason prince...,"[relationship, redflag, love, advice, fairytale]"
275,1209,2021-11-19 17:30:00+00:00,1461748607120461830,🚩Be on the lookout for #accountingfraud red fl...,SurgentAFE,"[accountingfraud, Surgent, fraud, financialsta...",lookout red flag offer content assess risk aud...,"[accountingfraud, surgent, fraud, financialsta..."
348,1282,2021-11-17 15:00:41+00:00,1460986255013990405,Don't do it to fit in do it because your finan...,hjmmanagement,"[hjmman, hjmmanagement, redflag, trend, budget...",fit finance say,"[hjmman, hjmmanagement, redflag, trend, budget..."
...,...,...,...,...,...,...,...,...
4727,5661,2021-10-05 14:28:19+00:00,1445395431136059396,"Beware The Facebook ""Whistleblower""! She Is A ...",PixelOkie,"[DeleteFacebook, Facebook, facebookwhistleblow...",beware fraud plant demand censorship,"[deletefacebook, facebook, facebookwhistleblow..."
4882,5816,2021-09-29 04:28:33+00:00,1443070168389734400,Dating #redflag🚩: someone who doesn't support ...,Cafe24_ph,"[redflag, business, Cafe24, onlinebusiness]",date someone support support register,"[redflag, business, cafe24, onlinebusiness]"
5265,6199,2021-09-09 05:20:01+00:00,1435835363021438978,"Clients who tell you a job should be ""easy"", ""...",lutrov,"[redflag, webdev, webdesign, business]",client tell job easy simple quick rarely worth,"[redflag, webdev, webdesign, business]"
5271,6205,2021-09-08 17:50:13+00:00,1435661768328564737,@VivintHome on what planet do u send people to...,DavidJKelley,"[redflag, redflag, redflag, Fraud, SecurityRisk]",planet you send people someone house w appoint...,"[redflag, redflag, redflag, fraud, securityrisk]"


In [97]:
len(tweets_comm)

80

### Unique words in community

In [98]:
# Where are we working:
dataframe = tweets_comm.copy()

In [99]:
#initialize an empty dict  
unique_words = {} 
for row in dataframe.clean_text:
    for word in row.split(" "):
        #if the word is encountered for the first time add to dict as key and set its value to 0
        unique_words.setdefault(word,0)
        #increase the value (i.e the count) of the word by 1 every time it is encountered
        unique_words[word] += 1 # get unique words and their frequences

In [100]:
unique_words.pop("")

1

In [101]:
unique_words.pop("red")

23

In [102]:
unique_words.pop("flag")

24

In [103]:
word_nodes = pd.DataFrame.from_dict(unique_words, orient="index")
word_nodes.reset_index(inplace=True)
word_nodes["Label"] = word_nodes["index"]
word_nodes.rename(columns={"index":"Id", 0:"delete"},inplace=True)
word_nodes = word_nodes.drop(columns=['delete']) 

word_nodes

,Id,Label
0,watch,watch
1,beginning,beginning
2,relationship,relationship
3,read,read
4,full,full
...,...,...
332,confirm,confirm
333,join,join
334,anniversary,anniversary
335,hand,hand


In [104]:
uw_df = pd.DataFrame.from_dict(unique_words, orient='index').reset_index()
uw_df.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
uw_df.sort_values(by=['Count'], ascending=False, inplace=True)
uw = unique_words.keys()

### Words network

In [105]:
h = 0
network = {} 
network_key = 0
for index, row in dataframe.iterrows():
    #hashtags extracted from Tweet do not have the # sign in front of them but we will add it to differentiate hashtags from words
    combined_list = [word for word in str.split(row["clean_text"], " ") if word in uw]
    #itertool product creates Cartesian product of each element in the combined list (hastag + words, both UNIQUE)
    for pair in itertools.product(combined_list, combined_list):
        #exclude self-loops and count each pair only once because our graph is undirected and we do not take self-loops into account
        if pair[0]!=pair[1] and not(pair[::-1] in network):
            network.setdefault(pair, 0)
            network[pair] += 1 

#network = {key: value for (key, value) in network.items() if value >= h} #filter the network
network_df = pd.DataFrame.from_dict(network, orient="index")

In [106]:
network_df.reset_index(inplace=True) 
network_df.columns = ["pair","weight"] # undirected graph 

In [107]:
#to get weighted graph we need a list of 3-element tuplels (u,v,w) where u and v are nodes and w is a number representing weight
up_weighted = []
for edge in network:
    #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
    #if(network[edge])>1: 
    up_weighted.append((edge[0],edge[1],network[edge]))

G = nx.Graph()
G.add_weighted_edges_from(up_weighted)

In [108]:
print(len(G.nodes())) # number of nodes
print(len(G.edges()))

336
1759


In [36]:
to_remove = list(set(list(word_nodes['Id'])) ^ set(list(G.nodes())))
for i in range(len(to_remove)):
    word_nodes = word_nodes.drop(word_nodes[word_nodes['Id'] == to_remove[i]].index)

Node_list save procedure for Gephi visualization:

In [37]:
path = "./DATA_COMM/"
comm = '12'

In [38]:
# SAVE NODE LIST
word_nodes.to_csv(path + "#nodelist_community"+comm+".csv", index=False) 

# We need to insert a column called "Modularity Class" and label
file = pd.read_csv(path + "#nodelist_community"+comm+".csv", sep = ",", header = 0, names = ['Id','Modularity Class'])
file.insert(loc = 1, column = 'Label', value = file['Id'])
file.to_csv(path + "#nodelist_community"+comm+".csv", index=False)

Edge_list saving procedure for Gephi visualization:

In [39]:
nx.write_weighted_edgelist(G, path + "#edgelist_community"+comm+".csv")

# We need to insert source, target, weight and type of edge
file = pd.read_csv(path + "#edgelist_community"+comm+".csv", sep = " ", header = 0, names = ['Source','Target','Weight'])
file.insert(loc = 2, column = 'Type', value = ['undirected']*len(file))
file.to_csv(path + "#edgelist_community"+comm+".csv", index=False)